In [455]:
import bz2
import numpy as np
import pandas as pd
import pickle
import scipy.stats as sp_stats
from itertools import product

sections = ['utj', 'isthmus', 'ia-junction', 'ampulla1', 'ampulla2']

classes2 = {'ordering':[('narrow_end','#30a2da'), ('wide_end','#fc4f30'), ('narrow_lumen','#e5ae38'),('wide_lumen','green')]}
classes2['utj'] = {'narrow_end':[1,2,3, 7,8, 13,21], 'wide_end':[4,5,6, 9,14,19], 'narrow_lumen':[15, 16, 17,18], 'wide_lumen':[10,11,12, 20]}
classes2['isthmus'] = {'narrow_end':[1,2,3,4,5,6], 'wide_end':[7,8, 9], 'narrow_lumen':[10, 11, 12, 13, 14, 15], 'wide_lumen':[16,17,18,19,20,21]}
classes2['ia-junction'] = {'narrow_end':[1,2,3,4,5,6,7,8], 'wide_end':[9,10,11,12,13,14,15,16], 'narrow_lumen':[17,18,19,20,21,22,23,24], 'wide_lumen':[25,26,27,28,29,30,31,32]}
classes2['ampulla1'] = {'narrow_end':[1,2,3,4,5,6], 'wide_end':[7,8,9,10, 11, 12], 'narrow_lumen':[13,14,15,16,17,18], 'wide_lumen':[19, 20, 21, 22, 23, 24]}
classes2['ampulla2'] = {'narrow_end':[1,2,3,4,5,6], 'wide_end':[7,8,9,10, 11, 12], 'narrow_lumen':[13,14,15,16,17,18], 'wide_lumen':[19, 20, 21, 22, 23, 24]}

# load the stats for the experiment
with bz2.BZ2File('./resources/analysis/output/cross_section_stats_v2d2_isth_amp.pickle.bz2', 'rb') as stats_file:
    stats_v2b = pickle.load(stats_file)
print(stats_v2b.keys())

v2b_data = None
for sec in sections:
    if sec in stats_v2b and 'data_frame' in stats_v2b[sec]:
        if v2b_data is None:
            v2b_data = stats_v2b[sec]['data_frame']
        else:
            v2b_data = v2b_data.append(stats_v2b[sec]['data_frame'])

print('EVs in replicate 1 in Ampulla',v2b_data.query("section=='ampulla1' & replicate==1")['radius_um'].count())
print('EVs in replicate 1 in Isthmus',v2b_data.query("section=='isthmus' & replicate==1")['radius_um'].count())

dict_keys(['isthmus', 'ampulla1'])
EVs in replicate 1 in Ampulla 194
EVs in replicate 1 in Isthmus 1025


|Aims| Parametric | Non-parametric |
|-|-|-|
|Compare samples from 2 independent data sets|2 sample T-Test (Student's)|Mann-Whitney U|
|Compare samples from 2 paired data sets|paired Student T-test|Wilcoxon Signed-Rank|
|Compare samples from >2 indep. data sets|one-way ANOVA|Kruskal-Wallis H Test|
|Compare samples from >2 paired data sets|repeated measures ANOVA|Friedman|

## Statistics functions follow

In [157]:
"""
Kolmogorov-Smirnov (one of the most common methods?)
exact with continuous distributions, less precise with discrete distributions
ks_stat, ks_p = sp_stats.ks_2samp(d1, d2)

Mann Withney U test, needs n > 25
robust to violations of homogeneity of variance

Ranksums similar to mw, assumes continuous distributions. It does not handle ties between measurements in x and y, (MW does)
rank_stat, rank_p = sp_stats.ranksums(d1, d2)

Epps-Singleton does not assume a continuous distribution, needs n > 25

brunnermunzel does not test if both distributions are the same
"""

def test_significance_non_parametric(d1, n_d1, d2, n_d2, alpha=0.05, continuous_distribution=True, paired_samples=False):
    """
    Useful tests for paired and non-paired non-parametric samples.
    """
    if n_d1 < 25 or n_d2 < 25:
        print('n<25, using the Anderson-Darling test for k-samples,', end =' ')
        # returns statistc, critical values, and significance level
        ak_stat, ak_critical_values, ak_significance_level = sp_stats.anderson_ksamp([d1, d2])
        # is the test value greater than the critical value for 5%?
        if ak_stat > ak_critical_values[2]:
            print(f'significance level: {ak_significance_level:.4f}, Reject H0')
            return True, ak_significance_level
    else:
        p_normal1 = test_normality(d1)
        p_normal2 = test_normality(d2)
        
        if n_d1 > 100 and n_d2 > 100:
            print('N is sufficiently large, parametric test will take precedence')
            return test_significance_parametric(d1, n_d1, d2, n_d2, alpha, paired_samples)
        else:
            if continuous_distribution:
                if paired_samples:
                    # wilcoxon signed-rank
                    wsr_stat, wsr_p = sp_stats.wilcoxon(d1, d2)
                    print('| Wilcoxon signed-rank,', end=' ')
                    if wsr_p > alpha:
                        pass
                    else:
                        print(f'stat:{wsr_stat} p: {wsr_p:.4f}, Reject H0')
                        return True, mwu_p
                else:
                    # Mann-Whitney U
                    mwu_stat, mwu_p = sp_stats.mannwhitneyu(d1, d2)
                    print('| Mann-Whitney U,', end=' ')
                    if mwu_p > alpha:
                        pass
                    else:
                        print(f'stat:{mwu_stat} p: {mwu_p:.4f}, Reject H0')
                        return True, mwu_p
            else:
                epps_stat, epps_p = sp_stats.epps_singleton_2samp(d1, d2)
                print('| Epps singleton 2 samp,', end=' ')
                if epps_p > alpha:
                    pass # print(f'Fail to reject h0')
                else:
                    print(f'p:{epps_p:.4f}, Reject H0')
                    return True, epps_p
    
    print('N.S.')
    return False, None

def test_normality(data):
    """
    Only the result from D'Agostino and Pearson's test is reported
    """
    statistic, p = sp_stats.normaltest(data)
    
    print(f'normality test', f'p={p:.3f}' if p > 1E-3 else f'p={p:.3E}', end=' ')
    if p < 0.05:
        print('(sig. non-normal)', end=' ')
    return p

def test_variance_homogeneity(d1, d2, p_normal1, p_normal2):
    """
    Levene test is more robust for data with significant deviations from normality
    """
    if p_normal1 > 0.05 and p_normal2 > 0.05:
        bartlet_stat, p_var = sp_stats.bartlett(d1, d2)
        print('variance homogeneity test (bartlett)', f'p={p_var:.3E}' if p_var < 1E-3 else f'p={p_var:.3f}', end=' ')
    else:
        levene_stat, p_var = sp_stats.levene(d1, d2)
        print(f'variance homogeneity test (levene)', f'p={p_var:.3E}' if p_var < 1E-3 else f'p={p_var:.3f}', end=' ')
    return p_var

def test_significance_parametric(d1, n_d1, d2, n_d2, alpa=0.05, paired_samples=False, p_normal1=None, p_normal2=None):
    """
    Useful tests for paired and non-paired non-parametric samples.
    These tests assume coninuous distributions
    """
    if n_d1 < 100 and n_d2 < 100:
        if n_d1 < 20 or n_d2 < 20:
            print('WARNING Sample size is too-low, non-parametric test must be used instead.')
            return False, None
        else:
            print('Sample size is low 20 < n < 100')
    if p_normal1 is None:
        p_normal1 = test_normality(d1)
    if p_normal2 is None:
        p_normal2 = test_normality(d2)
    # test equal variances
    p_var = test_variance_homogeneity(d1, d2, p_normal1, p_normal2)

    if paired_samples:
        # paired samples t-test
        ttest_stat, ttest_p = sp_stats.ttest_rel(d1, d2)
        print('| t-test paired,', end=' ')
    else:
        # independent samples t-test, assumes identical variances
        # We need to set equal_var=False if samples have different sizes https://stackoverflow.com/a/22613361 to execute a welch's test
        ttest_stat, ttest_p = sp_stats.ttest_ind(d1, d2, equal_var=False if p_var > alpha and n_d1 != n_d2 else p_var > alpha)
        print('| t-test indep.,', end=' ')
        
    if ttest_p > alpha:
        print('N.S.')
        return False, None
    else:
        print(f'p={ttest_p:.4f}, Reject H0')
        return True, ttest_p

In [137]:
def most_repeating_significant_differences(significant_differences):
    # identify the most repeating significant differences
    reps = dict()
    for r in range(len(significant_differences)):
        #print('In repeat',r)
        for d in significant_differences[r]:
            s = tuple(d[:2])
            if s not in reps:
                reps[s] = []
            reps[s].append([r,d[2]])

    # rank the identified repeating pairs by their frequency they repeat
    sorted_by_repeats = {}
    for k, repeats in reversed(sorted(reps.items(), key=lambda t: len(t[1]))):
        if len(repeats) not in sorted_by_repeats:
            sorted_by_repeats[len(repeats)] = []
        sorted_by_repeats[len(repeats)].append((k, repeats))

    # display those significant in every repeat
    print('Significantly different pairs of ROI in more than one repeat:')
    for k,pairs in sorted_by_repeats.items():
        if k > 1:
            print('repeats:', k)
            for pair in pairs:
                print(pair[0], end=' ')
                for values in pair[1]:
                    print(f'rep:{values[0]}', f'p={values[1]:.4f},' if values[1] > 0.0001 else f'{values[1]:.2E},', end=' ')
                print()

In [161]:
def differences_between_same_class_rois_in_same_cross_section(section, alpha):
    print('H0: both samples were drawn from a population with the same distribution')
    # we reject h0 if there is evidence to suggest the samples were taken from two differt distributions
    counters = {'tested':0, 'skipped':0}
    
    significant_differences = []
    # identify those pairs with significant differences
    for rep in range(5):
        differences = []
        for k,colour in classes2['ordering']:
            print('class:', k, 'replicate:', rep)
            print(section, k)
            # compute the combinations of same class elements
            for roi_1, roi_2 in combinations(classes2[s1][k], 2):
                d1 = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={roi_1-1}")['radius_um']
                d2 = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={roi_2-1}")['radius_um']
                n_d1, n_d2 = len(d1), len(d2)

                print(f'roi {roi_1} ({n_d1}) - roi {roi_2} ({n_d2})', end=' ')
                if n_d1 < 2 or n_d2 < 2:
                    print('SKIPPING pair due to scarcity of observations')
                    counters['skipped'] += 1
                else:
                    significant, significance = test_significance_non_parametric(d1, n_d1, d2, n_d2, alpha, continuous_distribution=True, paired_samples=False)
                    counters['tested'] += 1
                    if significant:
                        differences.append([roi_1, roi_2, significance])
        significant_differences.append(differences)
    ts = len([e for l in significant_differences for e in l])
    print('Pairs tested:', counters['tested'], 'skipped:', counters['skipped'], 'significant:', ts, 'total:', counters['tested'] + counters['skipped'] + ts)
    return significant_differences

In [159]:
# I need to compare based on the pooled variance per class
def differences_between_non_same_class_rois_in_same_cross_section(section, alpha):
    print('H0: both samples were drawn from a population with the same distribution')
    # we reject h0 if there is evidence to suggest the samples were taken from two differt distributions
    counters = {'tested':0, 'skipped':0}
    
    significant_differences = []
    # identify those pairs with significant differences
    for rep in range(5):
        differences = []
        for k,colour in classes2['ordering']:
            print('class:', k, 'replicate:', rep)
            print(section, k)
            # compute the combinations of same class elements
            for roi_1, roi_2 in combinations(classes2[s1][k], 2):
                d1 = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={roi_1-1}")['radius_um']
                d2 = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={roi_2-1}")['radius_um']
                n_d1, n_d2 = len(d1), len(d2)

                print(f'roi {roi_1} ({n_d1}) - roi {roi_2} ({n_d2})', end=' ')
                if n_d1 < 2 or n_d2 < 2:
                    print('SKIPPING pair due to scarcity of observations')
                    counters['skipped'] += 1
                else:
                    significant, significance = test_significance_non_parametric(d1, n_d1, d2, n_d2, alpha, continuous_distribution=True, paired_samples=False)
                    counters['tested'] += 1
                    if significant:
                        differences.append([roi_1, roi_2, significance])
        significant_differences.append(differences)
    ts = len([e for l in significant_differences for e in l])
    print('Pairs tested:', counters['tested'], 'skipped:', counters['skipped'], 'significant:', ts, 'total:', counters['tested'] + counters['skipped'] + ts)
    return significant_differences

In [151]:
len([e for l in significant_differences for e in l])

16

In [162]:
# Differences between ROIs of the same class Isthmus
significant_differences = differences_between_same_class_rois_in_same_cross_section('isthmus', 0.05)

H0: both samples were drawn from a population with the same distribution
class: narrow_end replicate: 0
isthmus narrow_end
roi 1 (66) - roi 2 (72) normality test p=0.009 (sig. non-normal) normality test p=4.137E-07 (sig. non-normal) | Mann-Withney U, N.S.
roi 1 (66) - roi 3 (114) normality test p=0.009 (sig. non-normal) normality test p=1.523E-11 (sig. non-normal) | Mann-Withney U, N.S.
roi 1 (66) - roi 4 (70) normality test p=0.009 (sig. non-normal) normality test p=4.235E-04 (sig. non-normal) | Mann-Withney U, N.S.
roi 1 (66) - roi 5 (82) normality test p=0.009 (sig. non-normal) normality test p=6.738E-08 (sig. non-normal) | Mann-Withney U, N.S.
roi 1 (66) - roi 6 (61) normality test p=0.009 (sig. non-normal) normality test p=0.650 | Mann-Withney U, N.S.
roi 2 (72) - roi 3 (114) normality test p=4.137E-07 (sig. non-normal) normality test p=1.523E-11 (sig. non-normal) | Mann-Withney U, N.S.
roi 2 (72) - roi 4 (70) normality test p=4.137E-07 (sig. non-normal) normality test p=4.235E-04

C:\Users\carlos2\Miniconda3\envs\data_analysis_py36\lib\site-packages\ipykernel_launcher.py:24: UserWarning: p-value capped: true value larger than 0.25


 SKIPPING pair due to scarcity of observations
roi 19 (11) - roi 24 (0) SKIPPING pair due to scarcity of observations
roi 20 (13) - roi 21 (14) n<25, using the Anderson-Darling test for k-samples, N.S.
roi 20 (13) - roi 22 (0) SKIPPING pair due to scarcity of observations
roi 20 (13) - roi 23 (0) SKIPPING pair due to scarcity of observations
roi 20 (13) - roi 24 (0) SKIPPING pair due to scarcity of observations
roi 21 (14) - roi 22 (0) SKIPPING pair due to scarcity of observations
roi 21 (14) - roi 23 (0) SKIPPING pair due to scarcity of observations
roi 21 (14) - roi 24 (0) SKIPPING pair due to scarcity of observations
roi 22 (0) - roi 23 (0) SKIPPING pair due to scarcity of observations
roi 22 (0) - roi 24 (0) SKIPPING pair due to scarcity of observations
roi 23 (0) - roi 24 (0) SKIPPING pair due to scarcity of observations
class: narrow_end replicate: 1
isthmus narrow_end
roi 1 (66) - roi 2 (68) normality test p=2.766E-07 (sig. non-normal) normality test p=1.118E-06 (sig. non-normal

In [147]:
most_repeating_significant_differences(significant_differences)

Significantly different pairs of ROI in more than one repeat:
repeats: 2
(2, 6) rep:3 p=0.0038, rep:4 p=0.0128, 
(7, 8) rep:1 p=0.0171, rep:3 p=0.0432, 


In [148]:
# Differences between ROIs of the same class Ampulla
significant_differences = differences_between_same_class_rois_in_same_cross_section('ampulla1', 0.05)

H0: both samples were drawn from a population with the same distribution
class: narrow_end replicate: 0
ampulla1 narrow_end
roi 1 (16) - roi 2 (13) n<25, using the Anderson-Darling test for k-samples N.S.
roi 1 (16) - roi 3 (14) n<25, using the Anderson-Darling test for k-samples N.S.
roi 1 (16) - roi 4 (15) n<25, using the Anderson-Darling test for k-samples N.S.
roi 1 (16) - roi 5 (12) n<25, using the Anderson-Darling test for k-samples N.S.
roi 1 (16) - roi 6 (13) n<25, using the Anderson-Darling test for k-samples N.S.
roi 2 (13) - roi 3 (14) n<25, using the Anderson-Darling test for k-samples significance level: 0.0412, Reject H0
roi 2 (13) - roi 4 (15) n<25, using the Anderson-Darling test for k-samples N.S.
roi 2 (13) - roi 5 (12) n<25, using the Anderson-Darling test for k-samples N.S.
roi 2 (13) - roi 6 (13) n<25, using the Anderson-Darling test for k-samples N.S.
roi 3 (14) - roi 4 (15) n<25, using the Anderson-Darling test for k-samples N.S.
roi 3 (14) - roi 5 (12) n<25, usi

C:\Users\carlos2\Miniconda3\envs\data_analysis_py36\lib\site-packages\ipykernel_launcher.py:24: UserWarning: p-value capped: true value larger than 0.25


N.S.
roi 7 (7) - roi 11 (9) n<25, using the Anderson-Darling test for k-samples N.S.
roi 7 (7) - roi 12 (7) n<25, using the Anderson-Darling test for k-samples N.S.
roi 8 (5) - roi 9 (11) n<25, using the Anderson-Darling test for k-samples N.S.
roi 8 (5) - roi 10 (4) n<25, using the Anderson-Darling test for k-samples significance level: 0.0255, Reject H0
roi 8 (5) - roi 11 (9) n<25, using the Anderson-Darling test for k-samples N.S.
roi 8 (5) - roi 12 (7) n<25, using the Anderson-Darling test for k-samples N.S.
roi 9 (11) - roi 10 (4) n<25, using the Anderson-Darling test for k-samples N.S.
roi 9 (11) - roi 11 (9) n<25, using the Anderson-Darling test for k-samples N.S.
roi 9 (11) - roi 12 (7) n<25, using the Anderson-Darling test for k-samples N.S.
roi 10 (4) - roi 11 (9) n<25, using the Anderson-Darling test for k-samples N.S.
roi 10 (4) - roi 12 (7) n<25, using the Anderson-Darling test for k-samples N.S.
roi 11 (9) - roi 12 (7) n<25, using the Anderson-Darling test for k-samples N

In [149]:
most_repeating_significant_differences(significant_differences)

Significantly different pairs of ROI in more than one repeat:
repeats: 2
(10, 11) rep:1 p=0.0292, rep:3 p=0.0380, 


In [ ]:
# Differences between ROIs of distinct classes in the Isthmus
significant_differences = differences_between_non_same_class_rois_in_same_cross_section('isthmus', 0.05)

In [21]:
# Differences between Isth-Amp ROIs of the same category
s1 = 'isthmus'
s2 = 'ampulla1'
print('H0: both samples were drawn from a population with the same distribution')
alpha = 0.05
significant_differences = []

# identify those pairs with significant differences
for rep in range(5):
    differences = []
    for k,c in classes2['ordering']:
        print('section:', k, 'replicate:', rep)
        print('Isthmus-Ampulla')
        for i_roi, a_roi in product(classes2[s1][k], classes2[s2][k]):
            d_i = v2b_data.query(f"section=='{s1}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
            d_a = v2b_data.query(f"section=='{s2}' & replicate=={rep} & roi=={a_roi-1}")['radius_um']
            n_d_i, n_d_a= len(d_i), len(d_a)
            # Mann Withney U test, h0: assumption that both samples were drawn from a population with the same distribution
            # we reject h0 if there is evidence to suggest the samples were taken from two differt distributions
            x2, mwu_p = sp_stats.mannwhitneyu(d_i, d_a)
            x22, rank_p = sp_stats.ranksums(d_i, d_a)
            epps_stat, epps_p = sp_stats.epps_singleton_2samp(d_i, d_a)
            ks_stat, ks_p = sp_stats.ks_2samp(d_i, d_a)
            bm_stat, bm_p = sp_stats.brunnermunzel(d_i, d_a)

            print(f'roi {i_roi} ({n_d_i}) - roi {a_roi} ({n_d_a}), Mann-Withney U p: {mwu_p:.4f}, rank sums p: {rank_p:.4f}, Epps singleton 2 samp p: {epps_p:.4f}, KS 2 samp p: {ks_p:.4f}, Brunner-Munzel p:{bm_p:.4f}', end=' ')
            
            if mwu_p > alpha:
                print()
                pass
            else:
                differences.append([i_roi, a_roi, mwu_p])
                print(f'Reject h0')
    significant_differences.append(differences)

# identify the most repeating significant differences
reps = dict()
for r in range(len(significant_differences)):
    #print('In repeat',r)
    for d in significant_differences[r]:
        s = tuple(d[:2])
        if s not in reps:
            reps[s] = []
        reps[s].append(r)

H0: both samples were drawn from a population with the same distribution
section: narrow_end replicate: 0
Isthmus-Ampulla
roi 1 (12) - roi 1 (271), Mann-Withney U p: 0.4534, rank sums p: 0.9053, Epps singleton 2 samp p: 0.0408, KS 2 samp p: 0.9871, Brunner-Munzel p:0.8983 
roi 1 (12) - roi 2 (133), Mann-Withney U p: 0.3386, rank sums p: 0.6746, Epps singleton 2 samp p: 0.0480, KS 2 samp p: 0.9383, Brunner-Munzel p:0.6424 
roi 1 (12) - roi 3 (160), Mann-Withney U p: 0.4108, rank sums p: 0.8193, Epps singleton 2 samp p: 0.0010, KS 2 samp p: 0.9317, Brunner-Munzel p:0.7988 
roi 1 (12) - roi 4 (198), Mann-Withney U p: 0.4034, rank sums p: 0.8049, Epps singleton 2 samp p: 0.0050, KS 2 samp p: 0.9764, Brunner-Munzel p:0.7907 
roi 1 (12) - roi 5 (119), Mann-Withney U p: 0.1212, rank sums p: 0.2408, Epps singleton 2 samp p: 0.0034, KS 2 samp p: 0.6956, Brunner-Munzel p:0.2105 
roi 1 (12) - roi 6 (87), Mann-Withney U p: 0.3678, rank sums p: 0.7315, Epps singleton 2 samp p: 0.0002, KS 2 samp p: 

## Rois in same class

In [454]:
# rois in same class, all repeats together
def differences_between_rois_in_same_class_same_region(section):
    print('=='*20, '\n','ROIs in same class in', section)
    for class_name, class_colour in classes2['ordering']:
        print('Rois in', class_name, [roi for roi in classes2[section][class_name]])
        
        #for rep in range(5):
        #print('    repeat',rep+1, end=') ')
        dpr = []
        lengths = []
        for i_roi in classes2[section][class_name]:
            #d = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
            d = v2b_data.query(f"section=='{section}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths.append((i_roi, len(d)))
            dpr.append(d)
        min_n = min([len(d) for d in dpr])
        # samples with less than 3 data points cannot be tested
        if min_n < 3:
            print(f'min_n ({min_n}) < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr = dpr
            dpr = [d for d in dpr if len(d)>3]
            if len(dpr) < 2:
                print('There are not sufficient data points for testing')
                break
            print('Elements skipped:', [pair for pair in lengths if pair[1] < 3], end='\n\t      ')
            min_n = min([len(d) for d in dpr])

        # one way t-test assumes homoscedasticity, we must check the samples have the same variance
        parametric = False
        if min_n > 99:
            print(f'min_n ({min_n}) > 100, Normality not essential, skipping test', end=' ')
            # same variances?
            levene_stat, levene_p = sp_stats.levene(*dpr)
            if levene_p > 0.05:
                parametric = True
                print('Fail to reject Homoscedasticity', end=' ')
            else:
                print('Significantly different variances.', end=' ')

        elif min_n > 25:
            print(f'25 < min_n ({min_n}) < 100,', end='\n\t')
            if min([test_normality(d) for d in dpr]) < 0.05:
                print('\n\tAt least one distribution is significantly non-normal.', end=' ')
            else:
                print('\n\tDistributions not significantly different from normal', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
        else:
            print(f'3 < min_n ({min_n}) < 25,', end=' ')

        if parametric:
            print('\n\tParametric test.\n\t', sp_stats.f_oneway(*dpr))
        else:
            print('\n\tNon-parametric test.\n\t', sp_stats.kruskal(*dpr))
        
        dprs = dpr
        pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
        print('Pooled deviation', pooled_dev,'\n-------')
                
differences_between_rois_in_same_class_same_region('ampulla1')
differences_between_rois_in_same_class_same_region('isthmus')

 ROIs in same class in ampulla1
Rois in narrow_end [1, 2, 3, 4, 5, 6]
25 < min_n (55) < 100,
	normality test p=2.872E-14 (sig. non-normal) normality test p=1.705E-06 (sig. non-normal) normality test p=1.024E-04 (sig. non-normal) normality test p=0.010 (sig. non-normal) normality test p=2.063E-04 (sig. non-normal) normality test p=0.005 (sig. non-normal) 
	At least one distribution is significantly non-normal. 
	Non-parametric test.
	 KruskalResult(statistic=2.8671469946093606, pvalue=0.7204588897997898)
Pooled deviation 0.10164438796586625 
-------
Rois in wide_end [7, 8, 9, 10, 11, 12]
3 < min_n (22) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=7.224399809787691, pvalue=0.20447917650232983)
Pooled deviation 0.09937201741476544 
-------
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
3 < min_n (21) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=1.0951460331021663, pvalue=0.9545324225306633)
Pooled deviation 0.10182663018251106 
-------
Rois in wide_lumen [19, 20, 21, 22

In [253]:
# rois in same class, per repeat
def differences_between_rois_in_same_class_same_region(section):
    print('=='*20, '\n','ROIs in same class in', section)
    for class_name, class_colour in classes2['ordering']:
        print('Rois in', class_name, [roi for roi in classes2[section][class_name]])
        
        for rep in range(5):
            print('    repeat',rep+1, end=') ')
            dpr = []
            lengths = []
            for i_roi in classes2[section][class_name]:
                d = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                #d = v2b_data.query(f"section=='{section}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
                lengths.append((i_roi, len(d)))
                dpr.append(d)
            min_n = min([len(d) for d in dpr])
            # samples with less than 3 data points cannot be tested
            if min_n < 3:
                print(f'min_n ({min_n}) < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr = dpr
                dpr = [d for d in dpr if len(d)>3]
                if len(dpr) < 2:
                    print('There are not sufficient data points for testing')
                    break
                print('Elements skipped:', [pair for pair in lengths if pair[1] < 3], end='\n\t      ')
                min_n = min([len(d) for d in dpr])

            # one way t-test assumes homoscedasticity, we must check the samples have the same variance
            parametric = False
            if min_n > 99:
                print(f'min_n ({min_n}) > 100, Normality not essential, skipping test', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')

            elif min_n > 25:
                print(f'25 < min_n ({min_n}) < 100,', end='\n\t')
                if min([test_normality(d) for d in dpr]) < 0.05:
                    print('\n\tAt least one distribution is significantly non-normal.', end=' ')
                else:
                    print('\n\tDistributions not significantly different from normal', end=' ')
                    # same variances?
                    levene_stat, levene_p = sp_stats.levene(*dpr)
                    if levene_p > 0.05:
                        parametric = True
                        print('Fail to reject Homoscedasticity', end=' ')
                    else:
                        print('Significantly different variances.', end=' ')
            else:
                print(f'3 < min_n ({min_n}) < 25,', end=' ')

            if parametric:
                print('\n\tParametric test.\n\t', sp_stats.f_oneway(*dpr))
            else:
                print('\n\tNon-parametric test.\n\t', sp_stats.kruskal(*dpr))
                
differences_between_rois_in_same_class_same_region('ampulla1')
differences_between_rois_in_same_class_same_region('isthmus')

 ROIs in same class in ampulla1
Rois in narrow_end [1, 2, 3, 4, 5, 6]
    repeat 1) 3 < min_n (12) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=6.264950609744168, pvalue=0.28128505839557183)
    repeat 2) 3 < min_n (7) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=6.812084813077343, pvalue=0.23499511957662034)
    repeat 3) 3 < min_n (10) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=6.145245569825527, pvalue=0.2923414009515786)
    repeat 4) 3 < min_n (13) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=5.322263481002435, pvalue=0.37782899461252856)
    repeat 5) 3 < min_n (8) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=3.99043691190736, pvalue=0.5507936269626696)
Rois in wide_end [7, 8, 9, 10, 11, 12]
    repeat 1) 3 < min_n (4) < 25, 
	Non-parametric test.
	 KruskalResult(statistic=5.922654256134874, pvalue=0.31381791122878894)
    repeat 2) min_n (2) < 3, Attempting testing a subset of the data. Elements skipped: [(8, 2)]
	      3 < min

## Rois in different class

In [355]:
[p for p in combinations([t[0] for t in classes2['ordering']], 2)]

[('narrow_end', 'wide_end'),
 ('narrow_end', 'narrow_lumen'),
 ('narrow_end', 'wide_lumen'),
 ('wide_end', 'narrow_lumen'),
 ('wide_end', 'wide_lumen'),
 ('narrow_lumen', 'wide_lumen')]

In [382]:
a = [np.random.rand(20), np.random.rand(22), np.random.rand(28)]
b = [np.random.rand(35), np.random.rand(29)]

In [385]:
sp_stats.f_oneway(*a, *b), sp_stats.kruskal(*a, *b)

(F_onewayResult(statistic=1.4929413282705002, pvalue=0.2080911689680226),
 KruskalResult(statistic=6.318128524417716, pvalue=0.17661657302852227))

In [384]:
min([test_normality(d) for d in a + b]) < 0.05

normality test p=0.185 normality test p=0.065 normality test p=4.775E-05 (sig. non-normal) normality test p=0.064 normality test p=0.029 (sig. non-normal) 

True

In [452]:
# rois in different class
def differences_between_rois_in_different_class_same_region(section):
    """
    Test for significant differences between distinct ROI classes by using all the elements in each class
    In order to test each class against the others, we need to use combinations of the classes
    """
    print('=='*20, '\n','ROIs in different class in', section)
    for roi_class_1, roi_class_2 in [p for p in combinations([t[0] for t in classes2['ordering']], 2)]:
        print('Rois in', roi_class_1, [roi for roi in classes2[section][roi_class_1]])
        print('Rois in', roi_class_2, [roi for roi in classes2[section][roi_class_2]])
        
        for rep in range(5):
            dpr1, dpr2 = [], []
            lengths1, lengths2 = [], []
            print('    repeat',rep+1, end=') ')
            for i_roi in classes2[section][roi_class_1]:
                d1 = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                lengths1.append((i_roi, len(d1)))
                dpr1.append(d1)
            for i_roi in classes2[section][roi_class_2]:
                d2 = v2b_data.query(f"section=='{section}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                lengths2.append((i_roi, len(d2)))
                dpr2.append(d2)

            # all the test must be done to both lists of data
            min_n1 = min([len(d) for d in dpr1])
            min_n2 = min([len(d) for d in dpr2])
            # samples with less than 3 data points cannot be tested
            if min_n1 < 3:
                print('min_n1 < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr1 = dpr1
                dpr1 = [d for d in dpr1 if len(d)>3]
                print('Elements skipped:', [pair for pair in lengths1 if pair[1] < 3], end='\n\t      ')
                if len(dpr1) < 2:
                    print('There are not sufficient data points for testing')
                    break
                min_n1 = min([len(d) for d in dpr1])
            if min_n2 < 3:
                print('min_n2 < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr2 = dpr2
                dpr2 = [d for d in dpr2 if len(d)>3]
                print('Elements skipped:', [pair for pair in lengths2 if pair[1] < 3], end='\n\t      ')
                if len(dpr2) < 2:
                    print('There are not sufficient data points for testing')
                    break
                min_n2 = min([len(d) for d in dpr2])

            # one way t-test assumes homoscedasticity, we must check the samples have the same variance
            parametric = False
            if min_n1 > 99 and min_n2 > 99:
                print(f'min_n1 ({min_n1}) > 100 and min_n2 ({min_n2}) > 99, Normality not highly important, skipping test', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
                    
            elif min_n1 > 25 and min_n2 > 25:
                print(f'25 < min_n1 ({min_n1}), min_n2 ({min_n2}) < 100,', end='\n\t')
                if min([test_normality(d) for d in dpr1 + dpr2]) < 0.05:
                    print('\n\tAt least one distribution is significantly non-normal.', end=' ')
                else:
                    print('\n\tDistributions not significantly different from normal', end=' ')
                    # same variances?
                    levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                    if levene_p > 0.05:
                        parametric = True
                        print('Fail to reject Homoscedasticity', end=' ')
                    else:
                        print('Significantly different variances.', end=' ')
            else:
                print(f'3 < min_n ({min_n}) < 25,', end=' ')
            
            if parametric:
                res = sp_stats.f_oneway(*dpr1, *dpr2)
                print(f'\n\tParametric test (one-way ANOVA). {res[1]:.4f}', end=' ')
            else:
                res = sp_stats.kruskal(*dpr1, *dpr2)
                print(f'\n\tNon-parametric test (Kruskal-Wallis). {res[1]:.4f}', end=' ')
            print('N.S.' if res[1] > alpha else 'Significant differences. Reject H0')
            dprs = dpr1 + dpr2
            pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
            print('Pooled deviation', pooled_dev,'\n-------')

differences_between_rois_in_different_class_same_region('ampulla1')
differences_between_rois_in_different_class_same_region('isthmus')

 ROIs in different class in ampulla1
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in wide_end [7, 8, 9, 10, 11, 12]
    repeat 1) 3 < min_n (6) < 25, 
	Non-parametric test. 0.3143 N.S.
Pooled deviation 0.10438681225140536 
-------
    repeat 2) min_n2 < 3, Attempting testing a subset of the data. Elements skipped: [(8, 2)]
	      3 < min_n (6) < 25, 
	Non-parametric test. 0.0202 Significant differences. Reject H0
Pooled deviation 0.09735700729758767 
-------
    repeat 3) 3 < min_n (6) < 25, 
	Non-parametric test. 0.3781 N.S.
Pooled deviation 0.10085970770105468 
-------
    repeat 4) 3 < min_n (6) < 25, 
	Non-parametric test. 0.2148 N.S.
Pooled deviation 0.09960375410949662 
-------
    repeat 5) 3 < min_n (6) < 25, 
	Non-parametric test. 0.4711 N.S.
Pooled deviation 0.10663371494757747 
-------
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
    repeat 1) 3 < min_n (6) < 25, 
	Non-parametric test. 0.5841 N.S.
Pooled deviation 0.10277910441401455 
---

In [453]:
def differences_between_rois_in_different_class_same_region_pooled(section):
    """
    Test for significant differences between distinct ROI classes by using all the elements in each class
    In order to test each class against the others, we need to use combinations of the classes
    """
    print('=='*20, '\n','ROIs in different class in', section)
    for roi_class_1, roi_class_2 in [p for p in combinations([t[0] for t in classes2['ordering']], 2)]:
        print('Rois in', roi_class_1, [roi for roi in classes2[section][roi_class_1]])
        print('Rois in', roi_class_2, [roi for roi in classes2[section][roi_class_2]])
        
        #for rep in range(5):
        dpr1, dpr2 = [], []
        lengths1, lengths2 = [], []
        #print('    repeat',rep+1, end=') ')
        for i_roi in classes2[section][roi_class_1]:
            d1 = v2b_data.query(f"section=='{section}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths1.append((i_roi, len(d1)))
            dpr1.append(d1)
        for i_roi in classes2[section][roi_class_2]:
            d2 = v2b_data.query(f"section=='{section}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths2.append((i_roi, len(d2)))
            dpr2.append(d2)

        # all the test must be done to both lists of data
        min_n1 = min([len(d) for d in dpr1])
        min_n2 = min([len(d) for d in dpr2])
        # samples with less than 3 data points cannot be tested
        if min_n1 < 3:
            print('min_n1 < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr1 = dpr1
            dpr1 = [d for d in dpr1 if len(d)>3]
            print('Elements skipped:', [pair for pair in lengths1 if pair[1] < 3], end='\n\t      ')
            if len(dpr1) < 2:
                print('There are not sufficient data points for testing')
                break
            min_n1 = min([len(d) for d in dpr1])
        if min_n2 < 3:
            print('min_n2 < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr2 = dpr2
            dpr2 = [d for d in dpr2 if len(d)>3]
            print('Elements skipped:', [pair for pair in lengths2 if pair[1] < 3], end='\n\t      ')
            if len(dpr2) < 2:
                print('There are not sufficient data points for testing')
                break
            min_n2 = min([len(d) for d in dpr2])

        # one way t-test assumes homoscedasticity, we must check the samples have the same variance
        parametric = False
        if min_n1 > 99 and min_n2 > 99:
            print(f'min_n1 ({min_n1}) > 100 and min_n2 ({min_n2}) > 99, Normality not highly important, skipping test', end=' ')
            # same variances?
            levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
            if levene_p > 0.05:
                parametric = True
                print('Fail to reject Homoscedasticity', end=' ')
            else:
                print('Significantly different variances.', end=' ')

        elif min_n1 > 25 and min_n2 > 25:
            print(f'25 < min_n1 ({min_n1}), min_n2 ({min_n2}) < 100,', end='\n\t')
            if min([test_normality(d) for d in dpr1 + dpr2]) < 0.05:
                print('\n\tAt least one distribution is significantly non-normal.', end=' ')
            else:
                print('\n\tDistributions not significantly different from normal', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
        else:
            print(f'3 < min_n ({min_n}) < 25,', end=' ')

        if parametric:
            res = sp_stats.f_oneway(*dpr1, *dpr2)
            print(f'\n\tParametric test. {res[1]:.4f}', end=' ')
        else:
            res = sp_stats.kruskal(*dpr1, *dpr2)
            print(f'\n\tNon-parametric test. {res[1]:.4f}', end=' ')
        print('N.S.' if res[1] > alpha else 'Significant differences. Reject H0')
        dprs = dpr1 + dpr2
        pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
        print('Pooled deviation', pooled_dev,'\n-------')
                
differences_between_rois_in_different_class_same_region_pooled('ampulla1')
differences_between_rois_in_different_class_same_region_pooled('isthmus')

 ROIs in different class in ampulla1
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in wide_end [7, 8, 9, 10, 11, 12]
3 < min_n (6) < 25, 
	Non-parametric test. 0.4553 N.S.
Pooled deviation 0.10100714042074604 
-------
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
3 < min_n (6) < 25, 
	Non-parametric test. 0.9689 N.S.
Pooled deviation 0.10171475249014589 
-------
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in wide_lumen [19, 20, 21, 22, 23, 24]
min_n2 < 3, Attempting testing a subset of the data. Elements skipped: [(20, 2)]
	      3 < min_n (6) < 25, 
	Non-parametric test. 0.7134 N.S.
Pooled deviation 0.10216002289982638 
-------
Rois in wide_end [7, 8, 9, 10, 11, 12]
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
3 < min_n (6) < 25, 
	Non-parametric test. 0.5769 N.S.
Pooled deviation 0.10088754253383027 
-------
Rois in wide_end [7, 8, 9, 10, 11, 12]
Rois in wide_lumen [19, 20, 21, 22, 23, 24]
min_n2 < 3, Attempting testing a subset of the data. Element

# Same class - different region

In [451]:
# rois in different class
def differences_between_rois_in_same_class_different_region():
    """
    Test for significant differences between distinct ROI classes by using all the elements in each class
    In order to test each class against the others, we need to use combinations of the classes
    """
    sections = ['isthmus', 'ampulla1']
    print('=='*20, '\n','ROIs in different class in', sections)
    for class_name, class_colour in classes2['ordering']:
        rois_class_1 = [roi for roi in classes2[sections[0]][class_name]]
        rois_class_2 = [roi for roi in classes2[sections[1]][class_name]]
        print('Rois in', class_name, rois_class_1)
        print('Rois in', class_name, rois_class_2)

        for rep in range(5):
            dpr1, dpr2 = [], []
            lengths1, lengths2 = [], []
            print('    repeat',rep+1, end=') ')
            for i_roi in rois_class_1:
                d1 = v2b_data.query(f"section=='{sections[0]}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                lengths1.append((i_roi, len(d1)))
                dpr1.append(d1)
            for i_roi in rois_class_2:
                d2 = v2b_data.query(f"section=='{sections[1]}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                lengths2.append((i_roi, len(d2)))
                dpr2.append(d2)

            # all the test must be done to both lists of data
            min_n1 = min([len(d) for d in dpr1])
            min_n2 = min([len(d) for d in dpr2])
            # samples with less than 3 data points cannot be tested
            if min_n1 < 3:
                print('min_n1 < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr1 = dpr1
                dpr1 = [d for d in dpr1 if len(d)>3]
                print('Elements skipped:', [pair for pair in lengths1 if pair[1] < 3], end='\n\t      ')
                if len(dpr1) < 2:
                    print('There are not sufficient data points for testing')
                    break
                min_n1 = min([len(d) for d in dpr1])
            if min_n2 < 3:
                print('min_n2 < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr2 = dpr2
                dpr2 = [d for d in dpr2 if len(d)>3]
                print('Elements skipped:', [pair for pair in lengths2 if pair[1] < 3], end='\n\t      ')
                if len(dpr2) < 2:
                    print('There are not sufficient data points for testing')
                    break
                min_n2 = min([len(d) for d in dpr2])

            # one way t-test assumes homoscedasticity, we must check the samples have the same variance
            parametric = False
            if min_n1 > 99 and min_n2 > 99:
                print(f'min_n1 ({min_n1}) > 100 and min_n2 ({min_n2}) > 99, Normality not highly important, skipping test', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
                    
            elif min_n1 > 25 and min_n2 > 25:
                print(f'25 < min_n1 ({min_n1}), min_n2 ({min_n2}) < 100,', end='\n\t')
                if min([test_normality(d) for d in dpr1 + dpr2]) < 0.05:
                    print('\n\tAt least one distribution is significantly non-normal.', end=' ')
                else:
                    print('\n\tDistributions not significantly different from normal', end=' ')
                    # same variances?
                    levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                    if levene_p > 0.05:
                        parametric = True
                        print('Fail to reject Homoscedasticity', end=' ')
                    else:
                        print('Significantly different variances.', end=' ')
            else:
                print(f'3 < min_n ({min_n}) < 25,', end=' ')
            
            if parametric:
                res = sp_stats.f_oneway(*dpr1, *dpr2)
                print(f'\n\tParametric test. {res[1]:.4f}', end=' ')
            else:
                res = sp_stats.kruskal(*dpr1, *dpr2)
                print(f'\n\tNon-parametric test. {res[1]:.4f}', end=' ')
            print('N.S.' if res[1] > alpha else 'Significant differences. Reject H0')
            
            dprs = dpr1 + dpr2
            pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
            print('Pooled deviation', pooled_dev,'\n-------')
                
differences_between_rois_in_same_class_different_region()

 ROIs in different class in ['isthmus', 'ampulla1']
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in narrow_end [1, 2, 3, 4, 5, 6]
    repeat 1) 3 < min_n (6) < 25, 
	Non-parametric test. 0.5844 N.S.
Pooled deviation 0.10165063601091319 
-------
    repeat 2) 3 < min_n (6) < 25, 
	Non-parametric test. 0.2672 N.S.
Pooled deviation 0.10099811037586647 
-------
    repeat 3) 3 < min_n (6) < 25, 
	Non-parametric test. 0.6171 N.S.
Pooled deviation 0.09899748154500136 
-------
    repeat 4) 3 < min_n (6) < 25, 
	Non-parametric test. 0.2487 N.S.
Pooled deviation 0.10263574499422072 
-------
    repeat 5) 3 < min_n (6) < 25, 
	Non-parametric test. 0.0663 N.S.
Pooled deviation 0.10145140638650826 
-------
Rois in wide_end [7, 8, 9]
Rois in wide_end [7, 8, 9, 10, 11, 12]
    repeat 1) 3 < min_n (6) < 25, 
	Non-parametric test. 0.4726 N.S.
Pooled deviation 0.10224048193164795 
-------
    repeat 2) min_n2 < 3, Attempting testing a subset of the data. Elements skipped: [(8, 2)]
	      3 < min_n (6) <

In [450]:
# rois in different class
def differences_between_rois_in_same_class_different_region_pooled():
    """
    Test for significant differences between distinct ROI classes by using all the elements in each class
    In order to test each class against the others, we need to use combinations of the classes
    """
    sections = ['isthmus', 'ampulla1']
    print('=='*20, '\n','ROIs in different class in', sections)
    for class_name, class_colour in classes2['ordering']:
        rois_class_1 = [roi for roi in classes2[sections[0]][class_name]]
        rois_class_2 = [roi for roi in classes2[sections[1]][class_name]]
        print('Rois in', class_name, rois_class_1)
        print('Rois in', class_name, rois_class_2)

        #for rep in range(5):
        dpr1, dpr2 = [], []
        lengths1, lengths2 = [], []
        #print('    repeat',rep+1, end=') ')
        for i_roi in rois_class_1:
            d1 = v2b_data.query(f"section=='{sections[0]}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths1.append((i_roi, len(d1)))
            dpr1.append(d1)
        for i_roi in rois_class_2:
            d2 = v2b_data.query(f"section=='{sections[1]}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths2.append((i_roi, len(d2)))
            dpr2.append(d2)

        # all the test must be done to both lists of data
        min_n1 = min([len(d) for d in dpr1])
        min_n2 = min([len(d) for d in dpr2])
        # samples with less than 3 data points cannot be tested
        if min_n1 < 3:
            print('min_n1 < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr1 = dpr1
            dpr1 = [d for d in dpr1 if len(d)>3]
            print('Elements skipped:', [pair for pair in lengths1 if pair[1] < 3], end='\n\t      ')
            if len(dpr1) < 2:
                print('There are not sufficient data points for testing')
                break
            min_n1 = min([len(d) for d in dpr1])
        if min_n2 < 3:
            print('min_n2 < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr2 = dpr2
            dpr2 = [d for d in dpr2 if len(d)>3]
            print('Elements skipped:', [pair for pair in lengths2 if pair[1] < 3], end='\n\t      ')
            if len(dpr2) < 2:
                print('There are not sufficient data points for testing')
                break
            min_n2 = min([len(d) for d in dpr2])

        # one way t-test assumes homoscedasticity, we must check the samples have the same variance
        parametric = False
        if min_n1 > 99 and min_n2 > 99:
            print(f'min_n1 ({min_n1}) > 100 and min_n2 ({min_n2}) > 99, Normality not highly important, skipping test', end=' ')
            # same variances?
            levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
            if levene_p > 0.05:
                parametric = True
                print('Fail to reject Homoscedasticity', end=' ')
            else:
                print('Significantly different variances.', end=' ')

        elif min_n1 > 25 and min_n2 > 25:
            print(f'25 < min_n1 ({min_n1}), min_n2 ({min_n2}) < 100,', end='\n\t')
            if min([test_normality(d) for d in dpr1 + dpr2]) < 0.05:
                print('\n\tAt least one distribution is significantly non-normal.', end=' ')
            else:
                print('\n\tDistributions not significantly different from normal', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
        else:
            print(f'3 < min_n ({min_n}) < 25,', end=' ')

        if parametric:
            res = sp_stats.f_oneway(*dpr1, *dpr2)
            print(f'\n\tParametric test. {res[1]:.4f}', end=' ')
        else:
            res = sp_stats.kruskal(*dpr1, *dpr2)
            print(f'\n\tNon-parametric test. {res[1]:.4f}', end=' ')
        print('N.S.' if res[1] > alpha else 'Significant differences. Reject H0')
        
        dprs = dpr1 + dpr2
        pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
        print('Pooled deviation', pooled_dev,'\n-------')
                
differences_between_rois_in_same_class_different_region_pooled()

 ROIs in different class in ['isthmus', 'ampulla1']
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in narrow_end [1, 2, 3, 4, 5, 6]
25 < min_n1 (324), min_n2 (55) < 100,
	normality test p=2.868E-49 (sig. non-normal) normality test p=2.158E-61 (sig. non-normal) normality test p=5.494E-86 (sig. non-normal) normality test p=4.870E-48 (sig. non-normal) normality test p=4.892E-29 (sig. non-normal) normality test p=1.042E-17 (sig. non-normal) normality test p=2.872E-14 (sig. non-normal) normality test p=1.705E-06 (sig. non-normal) normality test p=1.024E-04 (sig. non-normal) normality test p=0.010 (sig. non-normal) normality test p=2.063E-04 (sig. non-normal) normality test p=0.005 (sig. non-normal) 
	At least one distribution is significantly non-normal. 
	Non-parametric test. 0.8833 N.S.
Pooled deviation 0.10106137751956412 
-------
Rois in wide_end [7, 8, 9]
Rois in wide_end [7, 8, 9, 10, 11, 12]
3 < min_n (6) < 25, 
	Non-parametric test. 0.2587 N.S.
Pooled deviation 0.10201569047828363 
----

# Different class - different region

In [449]:
# rois in different class
def differences_between_rois_in_different_class_different_region():
    """
    Test for significant differences between same ROI classes in distinct region by using all the elements in each class
    In order to test each class against the others
    """
    sections = ['isthmus', 'ampulla1']
    print('=='*20, '\n','ROIs in different class in sections', sections)
    
    for roi_class1, roi_class2 in [p for p in combinations([t[0] for t in classes2['ordering']], 2)]:
        # fetch all the ROis in class x in the isthmus
        # fetch all the ROis in class x in the ampulla1
        # compare ROIs in both sectinons ysung 
        rois_class1 = [roi for roi in classes2[sections[0]][roi_class1]]
        rois_class2 = [roi for roi in classes2[sections[1]][roi_class2]]
        print('Rois in', roi_class1, rois_class1)
        print('Rois in', roi_class2, rois_class2)
        
        for rep in range(5):
            dpr1, dpr2 = [], []
            lengths1, lengths2 = [], []
            print('    repeat',rep+1, end=') ')
            for i_roi in rois_class1:
                d1 = v2b_data.query(f"section=='{sections[0]}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                lengths1.append((i_roi, len(d1)))
                dpr1.append(d1)
            for i_roi in rois_class2:
                d2 = v2b_data.query(f"section=='{sections[1]}' & replicate=={rep} & roi=={i_roi-1}")['radius_um']
                lengths2.append((i_roi, len(d2)))
                dpr2.append(d2)

            # all the test must be done to both lists of data
            min_n1 = min([len(d) for d in dpr1])
            min_n2 = min([len(d) for d in dpr2])
            # samples with less than 3 data points cannot be tested
            if min_n1 < 3:
                print('min_n1 < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr1 = dpr1
                dpr1 = [d for d in dpr1 if len(d)>3]
                print('Elements skipped:', [pair for pair in lengths1 if pair[1] < 3], end='\n\t      ')
                if len(dpr1) < 2:
                    print('There are not sufficient data points for testing')
                    break
                min_n1 = min([len(d) for d in dpr1])
            if min_n2 < 3:
                print('min_n2 < 3, Attempting testing a subset of the data.', end=' ')
                original_dpr2 = dpr2
                dpr2 = [d for d in dpr2 if len(d)>3]
                print('Elements skipped:', [pair for pair in lengths2 if pair[1] < 3], end='\n\t      ')
                if len(dpr2) < 2:
                    print('There are not sufficient data points for testing')
                    break
                min_n2 = min([len(d) for d in dpr2])

            # one way t-test assumes homoscedasticity, we must check the samples have the same variance
            parametric = False
            if min_n1 > 99 and min_n2 > 99:
                print(f'min_n1 ({min_n1}) > 100 and min_n2 ({min_n2}) > 99, Normality not highly important, skipping test', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
                    
            elif min_n1 > 25 and min_n2 > 25:
                print(f'25 < min_n1 ({min_n1}), min_n2 ({min_n2}) < 100,', end='\n\t')
                if min([test_normality(d) for d in dpr1 + dpr2]) < 0.05:
                    print('\n\tAt least one distribution is significantly non-normal.', end=' ')
                else:
                    print('\n\tDistributions not significantly different from normal', end=' ')
                    # same variances?
                    levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                    if levene_p > 0.05:
                        parametric = True
                        print('Fail to reject Homoscedasticity', end=' ')
                    else:
                        print('Significantly different variances.', end=' ')
            else:
                print(f'3 < min_n ({min_n}) < 25,', end=' ')
            
            if parametric:
                res = sp_stats.f_oneway(*dpr1, *dpr2)
                print('\n\tParametric test.\n\t', res[1], end=' ')
            else:
                res = sp_stats.kruskal(*dpr1, *dpr2)
                print('\n\tNon-parametric test.\n\t', res[1], end=' ')
            print('N.S.' if res[1] > alpha else 'Significant differences. Reject H0')
            
            dprs = dpr1 + dpr2
            pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
            print('Pooled deviation', pooled_dev,'\n-------')
                
differences_between_rois_in_different_class_different_region()

 ROIs in different class in sections ['isthmus', 'ampulla1']
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in wide_end [7, 8, 9, 10, 11, 12]
    repeat 1) 3 < min_n (6) < 25, 
	Non-parametric test.
	 0.6050700273474181 N.S.
Pooled deviation 0.10106819063342332 
-------
    repeat 2) min_n2 < 3, Attempting testing a subset of the data. Elements skipped: [(8, 2)]
	      3 < min_n (6) < 25, 
	Non-parametric test.
	 0.04980437824039922 Significant differences. Reject H0
Pooled deviation 0.10186359143744017 
-------
    repeat 3) 3 < min_n (6) < 25, 
	Non-parametric test.
	 0.5750102564597789 N.S.
Pooled deviation 0.09968846334659794 
-------
    repeat 4) 3 < min_n (6) < 25, 
	Non-parametric test.
	 0.0631075316539361 N.S.
Pooled deviation 0.10181095404872688 
-------
    repeat 5) 3 < min_n (6) < 25, 
	Non-parametric test.
	 0.12203668216076992 N.S.
Pooled deviation 0.10045778602538755 
-------
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
    repeat 1) 

In [448]:
# rois in different class
def differences_between_rois_in_different_class_different_region_pooled():
    """
    Test for significant differences between same ROI classes in distinct region by using all the elements in each class
    In order to test each class against the others
    """
    sections = ['isthmus', 'ampulla1']
    print('=='*20, '\n','ROIs in different class in sections', sections)
    
    for roi_class1, roi_class2 in [p for p in combinations([t[0] for t in classes2['ordering']], 2)]:
        # fetch all the ROis in class x in the isthmus
        # fetch all the ROis in class x in the ampulla1
        # compare ROIs in both sectinons ysung 
        rois_class1 = [roi for roi in classes2[sections[0]][roi_class1]]
        rois_class2 = [roi for roi in classes2[sections[1]][roi_class2]]
        print('Rois in', roi_class1, rois_class1)
        print('Rois in', roi_class2, rois_class2)
        
        #for rep in range(5):
        dpr1, dpr2 = [], []
        lengths1, lengths2 = [], []
        #print('    repeat',rep+1, end=') ')
        for i_roi in rois_class1:
            d1 = v2b_data.query(f"section=='{sections[0]}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths1.append((i_roi, len(d1)))
            dpr1.append(d1)
        for i_roi in rois_class2:
            d2 = v2b_data.query(f"section=='{sections[1]}' & replicate in [0,1,2,3,4] & roi=={i_roi-1}")['radius_um']
            lengths2.append((i_roi, len(d2)))
            dpr2.append(d2)

        # all the test must be done to both lists of data
        min_n1 = min([len(d) for d in dpr1])
        min_n2 = min([len(d) for d in dpr2])
        # samples with less than 3 data points cannot be tested
        if min_n1 < 3:
            print('min_n1 < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr1 = dpr1
            dpr1 = [d for d in dpr1 if len(d)>3]
            print('Elements skipped:', [pair for pair in lengths1 if pair[1] < 3], end='\n\t      ')
            if len(dpr1) < 2:
                print('There are not sufficient data points for testing')
                break
            min_n1 = min([len(d) for d in dpr1])
        if min_n2 < 3:
            print('min_n2 < 3, Attempting testing a subset of the data.', end=' ')
            original_dpr2 = dpr2
            dpr2 = [d for d in dpr2 if len(d)>3]
            print('Elements skipped:', [pair for pair in lengths2 if pair[1] < 3], end='\n\t      ')
            if len(dpr2) < 2:
                print('There are not sufficient data points for testing')
                break
            min_n2 = min([len(d) for d in dpr2])

        # one way t-test assumes homoscedasticity, we must check the samples have the same variance
        parametric = False
        if min_n1 > 99 and min_n2 > 99:
            print(f'min_n1 ({min_n1}) > 100 and min_n2 ({min_n2}) > 99, Normality not highly important, skipping test', end=' ')
            # same variances?
            levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
            if levene_p > 0.05:
                parametric = True
                print('Fail to reject Homoscedasticity', end=' ')
            else:
                print('Significantly different variances.', end=' ')

        elif min_n1 > 25 and min_n2 > 25:
            print(f'25 < min_n1 ({min_n1}), min_n2 ({min_n2}) < 100,', end='\n\t')
            if min([test_normality(d) for d in dpr1 + dpr2]) < 0.05:
                print('\n\tAt least one distribution is significantly non-normal.', end=' ')
            else:
                print('\n\tDistributions not significantly different from normal', end=' ')
                # same variances?
                levene_stat, levene_p = sp_stats.levene(*dpr1, *dpr2)
                if levene_p > 0.05:
                    parametric = True
                    print('Fail to reject Homoscedasticity', end=' ')
                else:
                    print('Significantly different variances.', end=' ')
        else:
            print(f'3 < min_n ({min_n}) < 25,', end=' ')

        if parametric:
            res = sp_stats.f_oneway(*dpr1, *dpr2)
            print('\n\tParametric test.\n\t', res[1], end=' ')
        else:
            res = sp_stats.kruskal(*dpr1, *dpr2)
            print('\n\tNon-parametric test.\n\t', res[1], end=' ')
        print('N.S.' if res[1] > alpha else 'Significant differences. Reject H0')
        
        dprs = dpr1 + dpr2
        pooled_dev = np.sqrt(np.sum([(r.count()-1) * (r.mean()**2) for r in dprs]) / (np.sum([r.count() for r in dprs]) - len(dprs)))
        print('Pooled deviation', pooled_dev,'\n-------')
differences_between_rois_in_different_class_different_region_pooled()

 ROIs in different class in sections ['isthmus', 'ampulla1']
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in wide_end [7, 8, 9, 10, 11, 12]
3 < min_n (6) < 25, 
	Non-parametric test.
	 0.4545985190003018 N.S.
Pooled deviation 0.10084631372962345 
-------
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
3 < min_n (6) < 25, 
	Non-parametric test.
	 0.9612330782875278 N.S.
Pooled deviation 0.10104445811582592 
-------
Rois in narrow_end [1, 2, 3, 4, 5, 6]
Rois in wide_lumen [19, 20, 21, 22, 23, 24]
min_n2 < 3, Attempting testing a subset of the data. Elements skipped: [(20, 2)]
	      3 < min_n (6) < 25, 
	Non-parametric test.
	 0.6940603588611964 N.S.
Pooled deviation 0.10106680383416455 
-------
Rois in wide_end [7, 8, 9]
Rois in narrow_lumen [13, 14, 15, 16, 17, 18]
3 < min_n (6) < 25, 
	Non-parametric test.
	 0.9705544365321911 N.S.
Pooled deviation 0.10239858946481938 
-------
Rois in wide_end [7, 8, 9]
Rois in wide_lumen [19, 20, 21, 22, 23, 24]
min_

# 1-way ANOVA
Steps
1. Find the mean for each of the groups.  
1. Find the overall mean (the mean of the groups combined).  

1. Find the Between Group Variation (**Treatments**): the deviation of each Group Mean from the Overall Mean. It is the variability due to interaction between the groups. AKA: Sum of Squares Between ($SS_B, SS_{Between}$), Sum of Squares of the Model ($SS_{Model}$), Sum of Squares Treatments ($SS_{Treatment}$) 
$$
\begin{equation*}
\mathbf{SS_{B}} = \frac{\sum \left( \sum \left( k_i \right) ^2 \right)}{n} - \frac{T^2}{N}
\end{equation*}
$$

1. Find the Within Group Variation (**Error**): the total deviation of each member’s score from the Group Mean. The variability in the data due to differences within people. AKA: Sum of Squares Within ($SS_{W}, SS_{Within}$), Sum of Squares Error ($SS_{Error}$, $SS_{E}$) or Sum of Squares Residual ($SS_{Residual}$) 
$$
\mathbf{SS_{W}} = \sum{Y^2} - \frac{\sum \left( \sum \left( a_i \right)^2 \right)}{n}   
$$

1. Find the F statistic: the ratio of Between Group Variation to Within Group Variation.
$$
\mathbf{F} = \frac{MS_{B}}{MS_{W}} \text{ or } \mathbf{F} = \frac{MS_{Treatment}}{MS_{Error}} \text{ or } \mathbf{F} = \frac{\text{expained variance}}{\text{unexplained variance}} \text{ or } \mathbf{F} = \frac{\text{between-group variability}}{\text{within-group variability}}
$$
where,
$$
\begin{align}
\mathbf{MS_{B}}=\frac{SS_{B}}{DF_{B}}, \mathbf{DF_{B}}=K-1 && \begin{cases} K & \text{number of groups (factors)}\end{cases}
\end{align}
$$
and
$$
\begin{align}
\mathbf{MS_{W}}=\frac{SS_{W}}{DF_{W}}, \mathbf{DF_{W}}=k(n-1) && \begin{cases} k & \text{level of factor} \\  n & \text{observ in group} \end{cases}
\end{align}
$$
Here, $MS_{Error}$ or $MS_{W}$ is the estimate of variance corresponding to $o^2$ of the model.

[source](https://jooskorstanje.com/1_Way_ANOVA_Pizza_Delivery.html), [compliment](https://www.marsja.se/four-ways-to-conduct-one-way-anovas-using-python/#_ANOVA_in_Python_using_SciPy)

In [326]:
A = [12.6, 12, 11.8, 11.9, 13, 12.5, 14]
B = [10, 10.2, 10, 12, 14, 13]
C = [10.1, 13, 13.4, 12.9, 8.9, 10.7, 13.6, 12]

all_scores = A + B + C
company_names = (['A'] * len(A)) +  (['B'] * len(B)) +  (['C'] * len(C))

In [349]:
test_data = pd.DataFrame({'company':company_names, 'score':all_scores})
test_data

,company,score
0,A,12.6
1,A,12.0
2,A,11.8
3,A,11.9
4,A,13.0
5,A,12.5
6,A,14.0
7,B,10.0
8,B,10.2
9,B,10.0


In [328]:
test_data.groupby('company').mean()

,score
company,
A,12.542857
B,11.533333
C,11.825000


In [329]:
# overall mean
overall_mean = test_data['score'].mean()
overall_mean

11.980952380952381

$latex SStotal = \sum Y^2 – \frac{T^2}{N}&s=2$

In [330]:
# compute sume of squares total
test_data['overall_mean'] = overall_mean
ss_total = sum((test_data['score'] - test_data['overall_mean'])**2)
ss_total

43.132380952380956

In [331]:
test_data.head()

,company,score,overall_mean
0,A,12.6,11.980952
1,A,12.0,11.980952
2,A,11.8,11.980952
3,A,11.9,11.980952
4,A,13.0,11.980952


In [332]:
# compute group means
group_means = test_data.groupby('company').mean()
group_means = group_means.rename(columns={'score':'group_mean'})
group_means

,group_mean,overall_mean
company,,
A,12.542857,11.980952
B,11.533333,11.980952
C,11.825000,11.980952


In [333]:
# add group means and overall means to the original DataFrame
test_data = test_data.merge(group_means, left_on='company', right_index=True)
test_data.head()

,company,score,overall_mean_x,group_mean,overall_mean_y
0,A,12.6,11.980952,12.542857,11.980952
1,A,12.0,11.980952,12.542857,11.980952
2,A,11.8,11.980952,12.542857,11.980952
3,A,11.9,11.980952,12.542857,11.980952
4,A,13.0,11.980952,12.542857,11.980952
5,A,12.5,11.980952,12.542857,11.980952
6,A,14.0,11.980952,12.542857,11.980952
7,B,10.0,11.980952,11.533333,11.980952
8,B,10.2,11.980952,11.533333,11.980952
9,B,10.0,11.980952,11.533333,11.980952


In [340]:
test_data = test_data.drop(columns=['overall_mean_y'])
test_data = test_data.rename(columns={'overall_mean_x':'overall_mean'})
test_data

,company,score,overall_mean,group_mean
0,A,12.6,11.980952,12.542857
1,A,12.0,11.980952,12.542857
2,A,11.8,11.980952,12.542857
3,A,11.9,11.980952,12.542857
4,A,13.0,11.980952,12.542857
5,A,12.5,11.980952,12.542857
6,A,14.0,11.980952,12.542857
7,B,10.0,11.980952,11.533333
8,B,10.2,11.980952,11.533333
9,B,10.0,11.980952,11.533333


In [341]:
# compute sum of squares residual, aka SSError
ss_residual = sum((test_data['score'] - test_data['group_mean'])**2)
ss_residual

39.52547619047619

In [342]:
# compute sum of squares model
ss_explained = sum((test_data['overall_mean'] - test_data['group_mean'])**2)
ss_explained

3.6069047619047776

In [403]:
# compute Mean Square Residual (this value is the pooled SD)
n_groups = len(set(test_data['company']))
n_obs = test_data.shape[0]
df_residual = n_obs - n_groups
ms_residual = ss_residual / df_residual
ms_residual, 'pooled SD:', ms_residual **2

(2.1958597883597886, 'pooled SD:', 4.821800210135495)

In [346]:
# compute Mean Square explained
df_explained = n_groups - 1
ms_explained = ss_explained / df_explained
ms_explained

1.8034523809523888

In [347]:
# compute F-values
f = ms_explained / ms_residual
f

0.8212966923081592

In [348]:
# compute p-value
import scipy.stats
p_value = 1 - scipy.stats.f.cdf(f, df_explained, df_residual)
p_value

0.4556832940515221

In [350]:
# and the easy way, using stats models
sp_stats.f_oneway(A, B, C)

F_onewayResult(statistic=0.8212966923081559, pvalue=0.4556832940515236)

In [416]:
sds = test_data.groupby('company')[['score']].std()
cnts = test_data.groupby('company')[['score']].count()

In [419]:
sds.columns = ['s_dev']

In [421]:
sds['counts'] = cnts

In [422]:
sds

,s_dev,counts
company,,
A,0.774289,7
B,1.728198,6
C,1.731845,8


In [433]:
np.sqrt(((sds['counts']-1) * sds['s_dev']**2).sum()/(sds['counts'].sum() - len(sds)))

1.481843375110807

In [430]:
groups = [[9.7, 2.5, 50],
[12.1, 2.9, 50],
[14.5, 3.2, 50],
[17.3, 6.8, 200]]
gfd = pd.DataFrame(groups, columns=['mean', 's_dev', 'counts'])
gfd

,mean,s_dev,counts
0,9.7,2.5,50
1,12.1,2.9,50
2,14.5,3.2,50
3,17.3,6.8,200


In [432]:
np.sqrt(((gfd['counts']-1) * gfd['s_dev']**2).sum()/(gfd['counts'].sum() - len(gfd)))

5.488258618082746

# The Pooled StDev

The pooled standard deviation is an estimate of the common standard deviation for all levels. The pooled standard deviation is the standard deviation of all data points around their group mean (not around the overall mean).  
Larger groups have a proportionally greater influence on the overall estimate of the pooled standard deviation. [minitab](https://support.minitab.com/en-us/minitab/18/help-and-how-to/modeling-statistics/anova/how-to/one-way-anova/interpret-the-results/all-statistics-and-graphs/means/)

Two-way ANOVA doesn't report this value - but it's easy to compute. The residual (or error) mean square reported on the ANOVA table is essentially a variance. We just need to compute its square root, which is the pooled SD.

This value can also be computed from stats per group as follows [source](https://www.statisticshowto.datasciencecentral.com/pooled-standard-deviation/),  
$$ SDpooled = \sqrt{\frac{\sum_{0,k} \left( (n_i-1)*sd_i^2 \right)}{\sum_{0,k} \left( n_i \right)- k}} $$

This formula computes correctly the values in these examples [1](https://support.minitab.com/en-us/minitab/18/help-and-how-to/statistics/basic-statistics/supporting-topics/data-concepts/what-is-the-pooled-standard-deviation/), [2](https://support.minitab.com/en-us/minitab/18/help-and-how-to/modeling-statistics/anova/how-to/one-way-anova/interpret-the-results/all-statistics-and-graphs/means/)

If we have the value per observation and the mean per factor, the formula is, [source](https://support.minitab.com/en-us/minitab/18/help-and-how-to/statistics/basic-statistics/supporting-topics/data-concepts/what-is-the-pooled-standard-deviation/) 
$$ SDpooled = \sqrt{\frac{\sum_i \left( \left(x_i- \bar{x}_{factor} \right)^2 \right)}{N-n_{factors}}} $$
Which produces the correct value for the second example in the source. This formula is the same used in computing the Mean Square Residual

In [398]:
# mean, sd, n (values per group)
groups = [[9.7, 2.5, 50],
[12.1, 2.9, 50],
[14.5, 3.2, 50],
[17.3, 6.8, 200]]

np.sqrt(np.sum([(r[2]-1) * (r[1]**2) for r in groups]) / (np.sum([r[2] for r in groups]) - len(groups)))

5.488258618082746

In [402]:
# value, factor, mean
values = [
    [18.95, 1, 14.5033],
    [12.62, 1, 14.5033],
    [11.94, 1, 14.5033],
    [14.42, 2, 10.5567],
    [10.06, 2, 10.5567],
    [7.19, 2, 10.5567]
]

np.sqrt(np.sum([(r[0] - r[2])**2 for r in values]) / (len(values) - len(set([r[1] for r in values]))))

3.7548945837400014